わたしたちはしっかりした基礎を築きました。--領域を表現すること、集計データを構築すること、関数を使いデータを変換すること、状態を作ること、並列性を利用すること。
手元の問題に対応する、より大きなコードの単位を構築しはじめるときです。
その大きな単位のことをコンポーネントといいます。
コードをコンポーネントに分割することで問題に対応した断片の中により高次元に考えることができる。(??)
コンポーネントの境界もまた、複数の開発者またはチームにコードのベースを分けるためのよい方法です。
それらもまた再利用のための機会です。

コンポーネントは(関数、レコード、プロトコルのような)よりきめ細かな、より大きな全体的な目的を持つ要素のコレクションです。
それらは呼び出し元が使うであろう外部のAPIを持っている。
それらは内部の実装もまた持っており、それはしばしばコンポーネントの状態を含んでいる。そして、データを並列に処理するために、またはイベントに反応するための処理をする分離したスレッドを作るために内的に並列性を使うことさえできます。(??)

Clojureの名前空間の中でアプリケーションの機能性を組織する方法を見ることでコンポーネントの考察を始めます。
これはコンポーネントの内部と外部の全てのコードに適用する。
次に、呼び出し元が使う外部のAPIを見ます。
これは関数呼び出しインターフェイスと長く生きる`core.async`の`channel`の使用の両方を考慮することを要求する。
最後に、すでに見てきた状態と並列性に対する道具を使って、コンポーネントの内部とコンポーネントの状態を管理することとそのライフ・サイクルを実装する方法を見ます。

第7章の「Compose Your Application」では完全なアプリケーションを組み立てるために、そのコンポーネントを次の段階に取ります。

# Organizing with Namespaces

Clojureのコードは一連の独立したトップ・レベルの形式(関数、レコード、プロトコルなど)としてコンパイルされ評価されますが、その独立した形式のグループに対してClojureは名前空間を提供します。
名前空間は命名され、集めるために使うことができる階層のコンテナ、整理し、形式のグループに名前を付ける。
名前空間のひとつの実用的な使用により、どんな所でも同じ名前が衝突することを心配せずに、コード内で簡潔な名前使うことが可能になります。
名前空間はわたしたちが意味するものを特定する手段を提供します。
Clojureのコードはよりきめ細かな要素から構成されているけれども、関数のレベルではなく名前空間のレベルに依存関係は宣言され、ロードされる。
各名前空間の中の`ns`マクロはその依存関係を定義し、依存関係のグラフをまとめて作ります。
この依存関係のグラフは名前空間がロードされる順序に影響します。
名前空間が提供する場合によっては、(両方とも型を指定する振舞いのオープンなシステムである)マルチメソッドかプロトコルのとある実装において、このロードの順番は重要でありえます。なぜならば、実装はそれが使用される前にかならずロードされるからです。

名前空間とコンポーネントの両方は整理するための道具です。
名前空間は関数を整理するための言語の特徴であり、コンポーネントは問題のレベルにおける整理する手段です。
これらの2つのアプローチは両方ともコードの構造を提供するために便利であり、連携して仕事をします。そして最終的に他の開発者が理解して使うことをより容易にします。

## Namespace Categories

Clojureにおいて、多くの理由から、ひと組の関数を名前空間の中にグループ化する。
以下の分類はアプリケーションに反映する論理的な名前空間の構築物を作るのに有用です。

- Utility (ユーティリティ)

    ユーティリティ名前空間は領域または目的によって整理された汎用の関数を提供する。
    例えば、文字列操作であったり特定のファイル形式をパースしたりするのための名前空間を作るでしょう。
    一般的に、ユーティリティ名前空間はあまり依存性を持たない。

- Data definition (データ定義)

    コレクションやエンティティを使うために名前空間の中で補助関数に加えてカスタム・コレクションや領域エンティティの組のそれぞれを定義することは一般的です。

- Abstraction (抽象化)

    プロトコルのような抽象化は名前空間の中で依存性が最小となるように分離される。

- Inplementation (実装)

    他の側面において、名前空間の中でプロトコルやインターフェイスによって定義される抽象化を実装することはしばしば有用です。

- Assembly (組み立て)

    実装の組と実装がどのように構築され、接続されるべきかを特定する構成を与えられたら、アセンブリ(組み立て)名前空間はすべてをひとつにつなげます。
    実装の内部では、一般的に抽象化(プロトコル)またはデータ構造だけが直接使われます。

- Entry point ()

    大部分のアプリケーションは(構成の集合を含む)アプリケーションの開始をアセンブリ(組み立て)の開始や他のライフ・サイクル操作につなぐひとつまたは複数のエントリ・ポイントを持つ。(??)

次の図はライブラリやアプリケーションにおいてその種の名前空間が一般的にどのように層をなしているかの一覧です。

![fig-06-01](fig_06_01.jpg)

この構造は自分自身で名前空間の構造を設計するときに有用なガイドラインです。
ユーティリティ名前空間は依存グラフの一番下にあり、それら自身にほとんど依存性がなく、複数の上の名前空間によって利用される。
次の層はデータまたは抽象化名前空間のどちらかからなり、アプリケーション自身のための構造ブロックを作る。
上の抽象化層は、それら抽象化の実装です。
その上のアセンブリ(組み立て)層では構成が処理され、実装が組み立てられつなげられ、アプリケーションの状態が作られる。
いちばん上はひとつまたは複数のエントリ・ポイントで、ウェブ・アプリやコマンド・ライン・インターフェイスやサービスなどになります。



## Public vs. Private Functions
# Designing Components APIs
## Manipulating Component Data with FUnctions
## Asynchronous APIs
# Connecting Components with Channels
## Fan Out(One-to-Many)
## Fan In (Many-to-One)
# Implementing Components
## Granularity of State
## Configuration
## Life Cycle
# Wrapping Up